# 선수 이름 매칭 분석 노트북

이 노트북은 VAEP 데이터와 통합 데이터의 선수 이름 일치도를 분석합니다.

## 분석 내용
1. VAEP 데이터에 players.json의 선수 이름 추가
2. 통합 데이터의 player와 비교하여 일치도 분석
3. 일치하지 않는 선수 목록 출력

## 1. 라이브러리 및 경로 설정

In [1]:
import pandas as pd
import json
from pathlib import Path
from difflib import SequenceMatcher
import unicodedata

# 파일 경로 설정
vaep_file = Path('../data/vaep_results/player_season_vaep_atomic.csv')
stats_file = Path('../data/player_match_stats/player_match_stats_combined_2017-18.csv')
players_json = Path('../data/wyscout/players.json')

## 2. 데이터 로드

In [2]:
# VAEP 데이터 로드
vaep_df = pd.read_csv(vaep_file)

# 컬럼명 정규화
column_mapping = {
    'player_id': 'playerId',
    'season_vaep_total': 'season_vaep_total',
    'avg_vaep_per_game': 'season_vaep_per_match',
    'total_actions': 'num_events',
    'num_games': 'matches_played',
    'vaep_per90': 'season_vaep_per90_avg'
}

for old_col, new_col in column_mapping.items():
    if old_col in vaep_df.columns:
        vaep_df = vaep_df.rename(columns={old_col: new_col})

# playerId=0 제외
vaep_df = vaep_df[vaep_df['playerId'] != 0].copy()

# 로드 결과 DataFrame
load_summary = pd.DataFrame({
    '항목': ['로드 완료 (총 행 수)', '컬럼 수', 'playerId=0 제외 후'],
    '값': [len(vaep_df) + (vaep_df['playerId'] == 0).sum(), len(vaep_df.columns), len(vaep_df)],
    '컬럼': [', '.join(list(vaep_df.columns)[:5]) + '...', str(len(vaep_df.columns)), '']
})
display(load_summary)

,항목,값,컬럼
0,로드 완료 (총 행 수),3030,"playerId, season_vaep_total, season_vaep_per_m..."
1,컬럼 수,6,6
2,playerId=0 제외 후,3030,


In [3]:
# 통합 Stats 데이터 로드
stats_df = pd.read_csv(stats_file)

# 로드 결과 DataFrame
stats_load_summary = pd.DataFrame({
    '항목': ['로드 완료 (총 행 수)', '컬럼 수', '고유 선수 수'],
    '값': [len(stats_df), len(stats_df.columns), stats_df['player'].nunique()]
})
display(stats_load_summary)

,항목,값
0,로드 완료 (총 행 수),50681
1,컬럼 수,36
2,고유 선수 수,2580


In [4]:
# players.json 로드 및 매핑 생성
with open(players_json, 'r', encoding='utf-8') as f:
    players_data = json.load(f)

# teams.json 로드 및 매핑 생성
teams_json = Path('../data/wyscout/teams.json')
with open(teams_json, 'r', encoding='utf-8') as f:
    teams_data = json.load(f)

# competitions.json 로드 및 월드컵/유로 챔피언스 ID 확인
competitions_json = Path('../data/wyscout/competitions.json')
with open(competitions_json, 'r', encoding='utf-8') as f:
    competitions_data = json.load(f)

# 제외할 competition ID (월드컵, 유로 챔피언스)
excluded_comp_ids = set()
for comp in competitions_data:
    comp_id = comp.get('wyId')
    comp_name = comp.get('name', '').lower()
    if 'world cup' in comp_name or 'european championship' in comp_name:
        excluded_comp_ids.add(comp_id)

print(f"제외할 Competition ID (월드컵/유로 챔피언스): {excluded_comp_ids}")

# matches 파일들에서 팀-competition 매핑 생성 (리그만)
# 월드컵/유로 챔피언스가 아닌 competition에 속한 팀만 포함
team_to_competitions = {}
matches_dir = Path('../data/wyscout')
matches_files = list(matches_dir.glob('matches_*.json'))

for matches_file in matches_files:
    file_name = matches_file.name
    # 월드컵/유로 챔피언스 파일은 건너뛰기
    if 'World_Cup' in file_name or 'European_Championship' in file_name:
        continue
    
    with open(matches_file, 'r', encoding='utf-8') as f:
        matches_data = json.load(f)
    
    for match in matches_data:
        comp_id = match.get('competitionId')
        # 제외할 competition이면 건너뛰기
        if comp_id in excluded_comp_ids:
            continue
        
        # 각 팀의 ID 수집
        if 'teamsData' in match:
            for team_id_str in match['teamsData'].keys():
                team_id = int(team_id_str)
                if team_id not in team_to_competitions:
                    team_to_competitions[team_id] = set()
                team_to_competitions[team_id].add(comp_id)

print(f"리그에 속한 팀 수: {len(team_to_competitions)}개")

# player_id -> 이름 매핑 생성
player_id_to_name = {}
player_id_to_nation = {}
player_id_to_team_id = {}

for player in players_data:
    player_id = player.get('wyId')
    if player_id:
        player_id = int(player_id)
        
        # 이름
        if 'shortName' in player:
            player_id_to_name[player_id] = player['shortName']
        elif 'firstName' in player and 'lastName' in player:
            player_id_to_name[player_id] = f"{player['firstName']} {player['lastName']}"
        
        # 국적
        if 'passportArea' in player and 'name' in player['passportArea']:
            player_id_to_nation[player_id] = player['passportArea']['name']
        
        # 팀 ID
        if 'currentTeamId' in player:
            player_id_to_team_id[player_id] = player['currentTeamId']

# 팀 ID -> 팀 이름 매핑 생성
team_id_to_name = {}
for team in teams_data:
    team_id = team.get('wyId')
    if team_id:
        team_id = int(team_id)
        if 'name' in team:
            team_id_to_name[team_id] = team['name']

# player_id -> 팀 이름 매핑 생성 (리그에 속한 팀만)
player_id_to_team_name = {}
for player_id, team_id in player_id_to_team_id.items():
    # 팀이 리그에 속하는지 확인 (월드컵/유로 챔피언스 제외)
    if team_id in team_to_competitions and team_id in team_id_to_name:
        player_id_to_team_name[player_id] = team_id_to_name[team_id]

# 매핑 결과 DataFrame
total_players_with_team = len(player_id_to_team_id)
league_players_with_team = len(player_id_to_team_name)
excluded_players = total_players_with_team - league_players_with_team

mapping_summary = pd.DataFrame({
    '매핑 유형': ['ID -> 이름', 'ID -> 국적', 'ID -> 팀 (전체)', 'ID -> 팀 (리그만)', '제외된 팀 (월드컵/유로)'],
    '수': [len(player_id_to_name), len(player_id_to_nation), total_players_with_team, league_players_with_team, excluded_players]
})
print("매핑 생성 완료 (월드컵/유로 챔피언스 제외)")
display(mapping_summary)

제외할 Competition ID (월드컵/유로 챔피언스): {28, 102}
리그에 속한 팀 수: 98개
매핑 생성 완료 (월드컵/유로 챔피언스 제외)


,매핑 유형,수
0,ID -> 이름,3603
1,ID -> 국적,3603
2,ID -> 팀 (전체),3603
3,ID -> 팀 (리그만),2439
4,제외된 팀 (월드컵/유로),1164


## 3. VAEP 데이터에 player_name 추가

In [5]:
# VAEP 데이터에 player_name, nation, team 추가
vaep_df['player_name'] = vaep_df['playerId'].map(player_id_to_name)
vaep_df['nation'] = vaep_df['playerId'].map(player_id_to_nation)
vaep_df['team'] = vaep_df['playerId'].map(player_id_to_team_name)

# 국가 이름을 국가 코드로 변환하는 매핑 (주요 국가 및 축구에서 자주 사용되는 국가)
country_name_to_code = {
    # 유럽 주요 국가
    'Spain': 'ESP', 'Italian': 'ITA', 'Italy': 'ITA',
    'France': 'FRA', 'Germany': 'GER', 'German': 'GER',
    'England': 'ENG', 'United Kingdom': 'ENG',
    'Portugal': 'POR', 'Netherlands': 'NED', 'Belgium': 'BEL',
    'Croatia': 'HRV', 'Serbia': 'SRB', 'Poland': 'POL',
    'Wales': 'WAL', 'Scotland': 'SCO', 'Northern Ireland': 'NIR',
    'Republic of Ireland': 'IRL', 'Ireland': 'IRL',
    'Denmark': 'DEN', 'Sweden': 'SWE', 'Norway': 'NOR',
    'Finland': 'FIN', 'Iceland': 'ISL', 'Switzerland': 'SUI',
    'Austria': 'AUT', 'Czech Republic': 'CZE', 'Slovakia': 'SVK',
    'Slovenia': 'SVN', 'Hungary': 'HUN', 'Romania': 'ROU',
    'Bulgaria': 'BUL', 'Greece': 'GRE', 'Turkey': 'TUR',
    'Russia': 'RUS', 'Ukraine': 'UKR', 'Belarus': 'BLR',
    'Albania': 'ALB', 'Bosnia and Herzegovina': 'BIH',
    'Montenegro': 'MNE', 'Macedonia': 'MKD', 'Kosovo': 'KVX',
    'Estonia': 'EST', 'Latvia': 'LVA', 'Lithuania': 'LTU',
    'Cyprus': 'CYP', 'Malta': 'MLT', 'Luxembourg': 'LUX',
    
    # 아프리카 주요 국가
    'Senegal': 'SEN', 'Ghana': 'GHA', 'Ivory Coast': 'CIV',
    "Côte d'Ivoire": 'CIV', "Cote d'Ivoire": 'CIV', 'Morocco': 'MAR', 'Algeria': 'ALG',
    'Tunisia': 'TUN', 'Egypt': 'EGY', 'Nigeria': 'NGA',
    'Cameroon': 'CMR', 'Congo DR': 'COD',
    'Democratic Republic of the Congo': 'COD',
    'Mali': 'MLI', 'Burkina Faso': 'BFA', 'Guinea': 'GUI',
    'Cape Verde': 'CPV', 'Cape Verde Islands': 'CPV', 'Gabon': 'GAB', 'Togo': 'TOG',
    'Benin': 'BEN', 'Angola': 'ANG', 'Mozambique': 'MOZ',  # AGO -> ANG로 통일
    'Zimbabwe': 'ZIM', 'Zambia': 'ZAM', 'Madagascar': 'MAD',
    'Mauritania': 'MTN', 'Guinea-Bissau': 'GNB',
    'Central African Republic': 'CAF', 'Burundi': 'BDI',
    'South Africa': 'RSA', 'Ethiopia': 'ETH', 'Kenya': 'KEN',
    'Uganda': 'UGA', 'Rwanda': 'RWA', 'Tanzania': 'TAN',
    
    # 아시아 주요 국가
    'Japan': 'JPN', 'South Korea': 'KOR', 'Korea Republic': 'KOR', 'Korea': 'KOR',
    'North Korea': 'PRK', 'Korea DPR': 'PRK', 'DPR Korea': 'PRK',
    'China': 'CHN', 'China PR': 'CHN',
    'Australia': 'AUS', 'New Zealand': 'NZL',
    'Saudi Arabia': 'SAU', 'United Arab Emirates': 'UAE',
    'Qatar': 'QAT', 'Iran': 'IRN', 'Iraq': 'IRQ',
    'Jordan': 'JOR', 'Lebanon': 'LBN', 'Syria': 'SYR',
    'Israel': 'ISR', 'Palestine': 'PLE',
    'Indonesia': 'IDN', 'Thailand': 'THA', 'Vietnam': 'VIE',
    'Philippines': 'PHI', 'Malaysia': 'MAS', 'Singapore': 'SIN',
    
    # 아메리카 주요 국가
    'Brazil': 'BRA', 'Argentina': 'ARG', 'Mexico': 'MEX',
    'United States': 'USA', 'Canada': 'CAN', 'Colombia': 'COL',
    'Chile': 'CHI', 'Uruguay': 'URU', 'Paraguay': 'PRY',
    'Ecuador': 'ECU', 'Peru': 'PER', 'Venezuela': 'VEN',
    'Costa Rica': 'CRC', 'Panama': 'PAN', 'Honduras': 'HON',
    'Jamaica': 'JAM', 'Trinidad and Tobago': 'TRI',
    
    # 기타
    'Armenia': 'ARM', 'Georgia': 'GEO', 'Azerbaijan': 'AZE',
    'Kazakhstan': 'KAZ', 'Uzbekistan': 'UZB',
    
    # 추가 매핑 (실제 데이터에서 발견된 케이스)
    'Curaçao': 'CUW', 'Curacao': 'CUW',
    'Equatorial Guinea': 'EQG', 'Guinea Equatorial': 'EQG',
    'São Tomé e Príncipe': 'STP', 'Sao Tome and Principe': 'STP',
    'Congo': 'COG',  # Republic of the Congo
    'Bosnia-Herzegovina': 'BIH', 'Bosnia and Herzegovina': 'BIH',
    'Croatia': 'CRO', 'HRV': 'CRO',  # CRO로 통일
    'Netherlands': 'NED', 'Holland': 'NED',
    'Switzerland': 'SUI',
    'United States': 'USA', 'US': 'USA',
    'Russia': 'RUS', 'Russian Federation': 'RUS',
    'Ukraine': 'UKR',
    'Turkey': 'TUR',
    'Greece': 'GRE',
    'Romania': 'ROU',
    'Czech Republic': 'CZE', 'Czechia': 'CZE',
    'Slovakia': 'SVK',
    'Slovenia': 'SVN',
    'Hungary': 'HUN',
    'Poland': 'POL',
    'Denmark': 'DEN',
    'Sweden': 'SWE',
    'Norway': 'NOR',
    'Finland': 'FIN',
    'Iceland': 'ISL',
    'Ireland': 'IRL', 'Republic of Ireland': 'IRL',
    'Wales': 'WAL',
    'Scotland': 'SCO',
    'Northern Ireland': 'NIR',
    'Mexico': 'MEX',
    'Colombia': 'COL',
    'Chile': 'CHI',
    'Uruguay': 'URU',
    'Paraguay': 'PRY',
    'Ecuador': 'ECU',
    'Peru': 'PER',
    'Venezuela': 'VEN',
    'Costa Rica': 'CRC',
    'Panama': 'PAN',
    'Honduras': 'HON',
    'Jamaica': 'JAM',
    'Trinidad and Tobago': 'TRI',
    'Morocco': 'MAR',
    'Tunisia': 'TUN',
    'Algeria': 'ALG',
    'Egypt': 'EGY',
    'Nigeria': 'NGA',
    'Ghana': 'GHA',
    'Senegal': 'SEN',
    'Ivory Coast': 'CIV', "Côte d'Ivoire": 'CIV',
    'Cameroon': 'CMR',
    'South Africa': 'RSA',
    'Japan': 'JPN',
    'South Korea': 'KOR',
    'Saudi Arabia': 'SAU',
    'Iran': 'IRN',
    'Iraq': 'IRQ',
    'United Arab Emirates': 'UAE',
    'Qatar': 'QAT',
    'Australia': 'AUS',
    'New Zealand': 'NZL'
}

# 국가 이름을 국가 코드로 변환
def convert_country_name_to_code(country_name):
    """국가 이름을 국가 코드로 변환"""
    if pd.isna(country_name):
        return None
    country_name = str(country_name).strip()
    
    # 이미 코드인 경우 (3자리 대문자)
    if len(country_name) == 3 and country_name.isupper():
        # 코드 매핑 확인 (예: HRV -> CRO)
        if country_name in country_name_to_code:
            return country_name_to_code[country_name]
        return country_name
    
    # 매핑에서 찾기
    if country_name in country_name_to_code:
        return country_name_to_code[country_name]
    
    # 대소문자 무시하고 찾기
    for name, code in country_name_to_code.items():
        if name.lower() == country_name.lower():
            return code
    
    # 부분 일치 시도 (예: "Côte d'Ivoire" vs "Cote d'Ivoire")
    country_name_lower = country_name.lower()
    for name, code in country_name_to_code.items():
        if name.lower().replace("'", "").replace("-", " ") == country_name_lower.replace("'", "").replace("-", " "):
            return code
    
    # 찾지 못하면 원본 반환 (이미 코드일 수 있음)
    return country_name

# VAEP 데이터의 nation을 국가 코드로 변환
vaep_df['nation'] = vaep_df['nation'].apply(convert_country_name_to_code)

# 팀 이름 매핑 (VAEP teams.json과 Stats 데이터 간 차이 해결)
team_name_mapping = {
    # 축약형/약어 매핑
    'AFC Bournemouth': 'Bournemouth',
    'Brighton & Hove Albion': 'Brighton',
    'Brighton and Hove Albion': 'Brighton',
    'Tottenham Hotspur': 'Tottenham',
    'West Ham United': 'West Ham',
    'Wolverhampton Wanderers': 'Wolves',
    'Leicester City': 'Leicester',
    'Newcastle United': 'Newcastle',
    'Manchester United': 'Man United',
    'Manchester City': 'Man City',
    'Crystal Palace': 'Crystal Palace',
    'Huddersfield Town': 'Huddersfield',
    'Swansea City': 'Swansea',
    'Stoke City': 'Stoke',
    'Watford': 'Watford',
    'Burnley': 'Burnley',
    'Southampton': 'Southampton',
    'Everton': 'Everton',
    'Liverpool': 'Liverpool',
    'Chelsea': 'Chelsea',
    'Arsenal': 'Arsenal',
    
    # 스페인 리그
    'Atlético Madrid': 'Atletico Madrid',
    'Atletico Madrid': 'Atletico Madrid',
    'Real Madrid': 'Real Madrid',
    'Barcelona': 'Barcelona',
    'Sevilla': 'Sevilla',
    'Valencia': 'Valencia',
    'Villarreal': 'Villarreal',
    'Real Betis': 'Real Betis',
    'Athletic Club': 'Athletic Bilbao',
    'Athletic Bilbao': 'Athletic Bilbao',
    'Real Sociedad': 'Real Sociedad',
    'Espanyol': 'Espanyol',
    'Celta de Vigo': 'Celta Vigo',
    'Celta Vigo': 'Celta Vigo',
    'Getafe': 'Getafe',
    'Levante': 'Levante',
    'Girona': 'Girona',
    'Eibar': 'Eibar',
    'Las Palmas': 'Las Palmas',
    'Málaga': 'Malaga',
    'Malaga': 'Malaga',
    'Deportivo La Coruña': 'Deportivo La Coruna',
    'Deportivo La Coruna': 'Deportivo La Coruna',
    'Deportivo la Coruña': 'Deportivo La Coruna',  # 소문자 변형
    'Deportivo la Coruna': 'Deportivo La Coruna',
    'Leganés': 'Leganes',
    'Leganes': 'Leganes',
    'Alavés': 'Alaves',
    'Alaves': 'Alaves',
    'Deportivo Alavés': 'Alaves',  # Deportivo Alavés -> Alaves로 통일
    'Deportivo Alaves': 'Alaves',
    
    # 이탈리아 리그
    'Juventus': 'Juventus',
    'Napoli': 'Napoli',
    'Roma': 'Roma',
    'Internazionale': 'Inter',
    'Inter': 'Inter',
    'Milan': 'Milan',
    'AC Milan': 'Milan',
    'Lazio': 'Lazio',
    'Fiorentina': 'Fiorentina',
    'Atalanta': 'Atalanta',
    'Torino': 'Torino',
    'Sampdoria': 'Sampdoria',
    'Udinese': 'Udinese',
    'Sassuolo': 'Sassuolo',
    'Genoa': 'Genoa',
    'Cagliari': 'Cagliari',
    'Chievo': 'Chievo',
    'Bologna': 'Bologna',
    'Hellas Verona': 'Verona',
    'Verona': 'Verona',
    'Benevento': 'Benevento',
    'Crotone': 'Crotone',
    'SPAL': 'SPAL',
    
    # 독일 리그
    'Bayern München': 'Bayern Munich',
    'Bayern Munich': 'Bayern Munich',
    'Borussia Dortmund': 'Dortmund',
    'Dortmund': 'Dortmund',
    'RB Leipzig': 'RB Leipzig',
    'Bayer Leverkusen': 'Bayer Leverkusen',
    'Borussia M\'gladbach': 'Borussia Monchengladbach',
    'Borussia Monchengladbach': 'Borussia Monchengladbach',
    'Mönchengladbach': 'Borussia Monchengladbach',
    'Schalke 04': 'Schalke',
    'Schalke': 'Schalke',
    'Hoffenheim': 'Hoffenheim',
    'Eintracht Frankfurt': 'Eintracht Frankfurt',
    'Wolfsburg': 'Wolfsburg',
    'Augsburg': 'Augsburg',
    'Hamburger SV': 'Hamburg',
    'Hamburg': 'Hamburg',
    'Mainz 05': 'Mainz',
    'Mainz': 'Mainz',
    'Hannover 96': 'Hannover',
    'Hannover': 'Hannover',
    'Stuttgart': 'Stuttgart',
    'Hertha Berlin': 'Hertha Berlin',
    'Werder Bremen': 'Werder Bremen',
    'Freiburg': 'Freiburg',
    'Köln': 'Koln',
    'Koln': 'Koln',
    
    # 프랑스 리그
    'Paris Saint-Germain': 'Paris Saint-Germain',
    'PSG': 'Paris Saint-Germain',
    'Monaco': 'Monaco',
    'Lyon': 'Lyon',
    'Marseille': 'Marseille',
    'Nice': 'Nice',
    'Bordeaux': 'Bordeaux',
    'Saint-Étienne': 'Saint-Etienne',
    'Saint-Etienne': 'Saint-Etienne',
    'Nantes': 'Nantes',
    'Lille': 'Lille',
    'Rennes': 'Rennes',
    'Toulouse': 'Toulouse',
    'Montpellier': 'Montpellier',
    'Dijon': 'Dijon',
    'Caen': 'Caen',
    'Angers': 'Angers',
    'Amiens SC': 'Amiens',
    'Amiens': 'Amiens',
    'Troyes': 'Troyes',
    'Strasbourg': 'Strasbourg',
    'Guingamp': 'Guingamp',
}

def normalize_team_name(team_name):
    """팀 이름 정규화 및 매핑"""
    if pd.isna(team_name):
        return ""
    team_name = str(team_name).strip()
    
    # 유니코드 문자를 ASCII로 변환 (é -> e, ü -> u 등)
    # NFD (Normalization Form Decomposed)로 분해한 후, combining characters 제거
    team_name = unicodedata.normalize('NFD', team_name)
    team_name = ''.join(c for c in team_name if unicodedata.category(c) != 'Mn')

    
    # 매핑에서 찾기
    if team_name in team_name_mapping:
        team_name = team_name_mapping[team_name]
    
    # 특수 케이스: "Deportivo Alavés" -> "Alaves"로 통일
    # "Deportivo la Coruña" -> "Deportivo La Coruna"로 통일
    team_name_lower = team_name.lower()
    if 'deportivo' in team_name_lower and 'alav' in team_name_lower:
        team_name = 'Alaves'
    elif 'deportivo' in team_name_lower and 'coru' in team_name_lower:
        team_name = 'Deportivo La Coruna'
    
    # 공백 정규화 및 소문자 변환
    team_name = ' '.join(team_name.split())
    return team_name.lower()

# VAEP 데이터의 팀 이름 정규화
vaep_df['team'] = vaep_df['team'].apply(lambda x: normalize_team_name(x) if pd.notna(x) else x)

# VAEP 데이터 변환 결과
vaep_summary = pd.DataFrame({
    '항목': ['이름 정보 있는 선수', '국적 정보 있는 선수'],
    '수': [vaep_df['player_name'].notna().sum(), vaep_df['nation'].notna().sum()],
    '전체': [len(vaep_df), len(vaep_df)],
    '비율 (%)': [
        vaep_df['player_name'].notna().sum()/len(vaep_df)*100,
        vaep_df['nation'].notna().sum()/len(vaep_df)*100
    ]
})
print("VAEP 데이터에 player_name 추가 및 국적 코드 변환 완료")
display(vaep_summary)

# 국적 코드 예시
nation_counts = vaep_df['nation'].value_counts().head(10).reset_index()
nation_counts.columns = ['국적 코드', '선수 수']
print("\n국적 코드 예시 (상위 10개)")
display(nation_counts)


VAEP 데이터에 player_name 추가 및 국적 코드 변환 완료


,항목,수,전체,비율 (%)
0,이름 정보 있는 선수,3030,3030,100.0
1,국적 정보 있는 선수,3030,3030,100.0



국적 코드 예시 (상위 10개)


,국적 코드,선수 수
0,ESP,420
1,ITA,349
2,FRA,247
3,GER,204
4,ENG,131
5,BRA,82
6,POR,62
7,SEN,55
8,CRO,51
9,SRB,47


## 4. 통합 데이터와 매칭하여 비교

In [6]:
# 통합 Stats 데이터와 VAEP 데이터 비교 (팀 기준 먼저, 그 다음 이름 기준)
# 1단계: 팀이 동일한지 확인
# 2단계: 팀이 동일한 선수들 중에서 이름이 일치하는지 확인
# 3단계: 통합

# 이름 정규화 함수 (비교용)
def normalize_name(name):
    """이름 정규화 (대소문자, 공백, 특수문자 처리)"""
    if pd.isna(name):
        return ""
    name = str(name).strip()
    # 공백 정규화
    name = ' '.join(name.split())
    return name.lower()

def normalize_team_name(team_name):
    """팀 이름 정규화"""
    if pd.isna(team_name):
        return ""
    team_name = str(team_name).strip()
    
    # 유니코드 문자를 ASCII로 변환 (é -> e, ü -> u 등)
    # NFD (Normalization Form Decomposed)로 분해한 후, combining characters 제거
    team_name = unicodedata.normalize('NFD', team_name)
    team_name = ''.join(c for c in team_name if unicodedata.category(c) != 'Mn')

    # 공백 정규화 및 소문자 변환
    team_name = ' '.join(team_name.split())
    return team_name.lower()

# Stats 데이터에서 선수별 집계 (이름, 국적, 팀, 경기 수)
# 팀별로 그룹화하기 위해 먼저 선수-팀 조합으로 집계
stats_player_team = stats_df.groupby(['player', 'team']).agg({
    'nation': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0],  # 가장 많이 나타난 국적
    'game': 'nunique'  # 경기 수
}).reset_index()
stats_player_team.columns = ['player', 'team', 'stats_nation', 'stats_games']

# VAEP 데이터 준비 (팀 정보 포함)
vaep_comparison = vaep_df[['playerId', 'player_name', 'nation', 'team', 'matches_played']].copy()
vaep_comparison.columns = ['playerId', 'vaep_name', 'vaep_nation', 'vaep_team', 'vaep_games']

# 팀과 이름이 모두 일치하는 선수들 찾기
comparison_results = []

for _, vaep_row in vaep_comparison.iterrows():
    vaep_name = vaep_row['vaep_name']
    vaep_team = vaep_row['vaep_team']
    
    if pd.isna(vaep_name):
        continue
    
    vaep_name_norm = normalize_name(vaep_name)
    vaep_team_norm = normalize_team_name(vaep_team) if pd.notna(vaep_team) else ""
    
    # 1단계: 팀이 동일한 선수들 찾기
    if vaep_team_norm:
        # Stats 데이터에서 같은 팀 찾기
        matching_teams = stats_player_team[
            stats_player_team['team'].apply(lambda x: normalize_team_name(x) == vaep_team_norm)
        ]
        
        if len(matching_teams) > 0:
            # 2단계: 팀이 동일한 선수들 중에서 이름이 일치하는지 확인
            for _, stats_row in matching_teams.iterrows():
                stats_name = stats_row['player']
                stats_name_norm = normalize_name(stats_name)
                
                if stats_name_norm == vaep_name_norm:
                    # 팀과 이름이 모두 일치하는 경우
                    comparison_results.append({
                        'playerId': vaep_row['playerId'],
                        'vaep_name': vaep_name,
                        'stats_name': stats_name,
                        'vaep_team': vaep_team,
                        'stats_team': stats_row['team'],
                        'vaep_nation': vaep_row['vaep_nation'] if pd.notna(vaep_row['vaep_nation']) else 'N/A',
                        'stats_nation': stats_row['stats_nation'] if pd.notna(stats_row['stats_nation']) else 'N/A',
                        'vaep_games': int(vaep_row['vaep_games']) if pd.notna(vaep_row['vaep_games']) else 'N/A',
                        'stats_games': int(stats_row['stats_games']) if pd.notna(stats_row['stats_games']) else 'N/A',
                        'team_match': True,
                        'name_match': True,
                        'nation_match': str(vaep_row['vaep_nation']).strip().upper() == str(stats_row['stats_nation']).strip().upper() if pd.notna(vaep_row['vaep_nation']) and pd.notna(stats_row['stats_nation']) else False
                    })

comparison_df = pd.DataFrame(comparison_results)

print("=" * 80)
print(f"팀과 이름이 모두 일치하는 선수 통합 결과 ({len(comparison_df)}명)")
print("=" * 80)

if len(comparison_df) > 0:
    print(f"\n총 통합된 선수 수: {len(comparison_df):,}명")
    print(f"팀 일치: {comparison_df['team_match'].sum():,}명 ({comparison_df['team_match'].sum()/len(comparison_df)*100:.1f}%)")
    print(f"이름 일치: {comparison_df['name_match'].sum():,}명 ({comparison_df['name_match'].sum()/len(comparison_df)*100:.1f}%)")
    print(f"국적 일치: {comparison_df['nation_match'].sum():,}명 ({comparison_df['nation_match'].sum()/len(comparison_df)*100:.1f}%)")
    print(f"국적 불일치: {(~comparison_df['nation_match']).sum():,}명 ({(~comparison_df['nation_match']).sum()/len(comparison_df)*100:.1f}%)")
    
    # 정렬 (국적 불일치 우선, 그 다음 이름 순)
    comparison_df = comparison_df.sort_values(['nation_match', 'vaep_name'], ascending=[True, True])
    
    print("\n" + "=" * 80)
    print("통합된 선수들의 상세 정보")
    print("=" * 80)
    print("\n형식: playerId | VAEP 이름 | Stats 이름 | VAEP 팀 | Stats 팀 | VAEP 국적 | Stats 국적 | VAEP 경기 | Stats 경기")
    print("-" * 120)
    
    for idx, (_, row) in enumerate(comparison_df.iterrows(), 1):
        player_id = int(row['playerId'])
        vaep_name = row['vaep_name']
        stats_name = row['stats_name']
        vaep_team = str(row['vaep_team'])[:25] if pd.notna(row['vaep_team']) else 'N/A'
        stats_team = str(row['stats_team'])[:25] if pd.notna(row['stats_team']) else 'N/A'
        vaep_nation = str(row['vaep_nation'])
        stats_nation = str(row['stats_nation'])
        vaep_games = str(row['vaep_games'])
        stats_games = str(row['stats_games'])
        nation_match_symbol = "✓" if row['nation_match'] else "✗"
        team_match_symbol = "✓" if row['team_match'] else "✗"
        
        print(f"{idx:4d}. playerId: {player_id:6d} | VAEP: {vaep_name:30s} | Stats: {stats_name:30s}")
        print(f"      팀: {vaep_team:25s} vs {stats_team:25s} {team_match_symbol} | 국적: {vaep_nation:5s} vs {stats_nation:5s} {nation_match_symbol} | 경기: {vaep_games:3s} vs {stats_games:3s}")
    
    print("\n" + "=" * 80)
    
    # 통합 데이터 생성
    print("\n통합 데이터 생성 중...")
    merged_df = vaep_df.merge(
        comparison_df[['playerId', 'stats_name', 'stats_team', 'stats_nation', 'stats_games']],
        on='playerId',
        how='left'
    )
    
    print(f"통합 완료: {len(merged_df):,} 행")
    print(f"통합된 선수: {merged_df['stats_name'].notna().sum():,}명")
    
else:
    print("\n통합된 선수가 없습니다.")
    print("팀 정보가 없는 VAEP 선수 수:", vaep_comparison['vaep_team'].isna().sum())
    print("VAEP 데이터에 팀 정보가 있는 선수 수:", vaep_comparison['vaep_team'].notna().sum())

print("\n" + "=" * 80)


팀과 이름이 모두 일치하는 선수 통합 결과 (167명)

총 통합된 선수 수: 167명
팀 일치: 167명 (100.0%)
이름 일치: 167명 (100.0%)
국적 일치: 148명 (88.6%)
국적 불일치: 19명 (11.4%)

통합된 선수들의 상세 정보

형식: playerId | VAEP 이름 | Stats 이름 | VAEP 팀 | Stats 팀 | VAEP 국적 | Stats 국적 | VAEP 경기 | Stats 경기
------------------------------------------------------------------------------------------------------------------------
   1. playerId:  40726 | VAEP: Allan                          | Stats: Allan                         
      팀: napoli                    vs Napoli                    ✓ | 국적: POR   vs BRA   ✗ | 경기: 38  vs 38 
   2. playerId:   3692 | VAEP: Bernardo Espinosa              | Stats: Bernardo Espinosa             
      팀: girona                    vs Girona                    ✓ | 국적: ESP   vs COL   ✗ | 경기: 34  vs 34 
   3. playerId: 222766 | VAEP: Bruno Gaspar                   | Stats: Bruno Gaspar                  
      팀: fiorentina                vs Fiorentina                ✓ | 국적: POR   vs ANG   ✗ | 경기: 15  vs 15 
   4. playerI

## 5. 이름은 같지만 팀/국적이 다른 선수 분석

### 호날두 (Cristiano Ronaldo) 팀 정보 출처 설명

**호날두의 팀 정보가 두 데이터 소스에서 다른 이유:**

1. **VAEP 데이터 (players.json) 출처:**
   - **팀**: Juventus (팀 ID: 3159)
   - **출처**: `players.json`의 `currentTeamId` 필드
   - **의미**: VAEP 데이터 수집 시점의 현재 소속 팀
   - **설명**: players.json은 시점에 따라 업데이트될 수 있으며, 호날두가 2018년 여름에 유벤투스로 이적했으므로, 데이터가 업데이트되었다면 `currentTeamId`가 Juventus로 설정됨

2. **Stats 데이터 (player_match_stats) 출처:**
   - **팀**: Real Madrid
   - **출처**: `player_match_stats_combined_2017-18.csv`
   - **의미**: 2017-18 시즌 실제 경기 데이터에서 나타난 팀
   - **설명**: 2017-18 시즌에 Real Madrid에서만 뛰었으므로 (27경기) Real Madrid만 나타남

3. **결론:**
   - **2017-18 시즌**: Real Madrid (Stats 데이터가 정확)
   - **2018-19 시즌 이후**: Juventus (VAEP players.json의 currentTeamId)
   - **차이점**: VAEP 데이터는 시점에 따라 업데이트된 현재 팀 정보를 포함하고, Stats 데이터는 특정 시즌의 실제 경기 데이터만 포함


In [7]:
# 이름은 같지만 팀이 다른 선수들, 그리고 이름은 같지만 국적이 다른 선수들 분석

# 이름 정규화 함수 (비교용)
def normalize_name(name):
    """이름 정규화 (대소문자, 공백, 특수문자 처리)"""
    if pd.isna(name):
        return ""
    name = str(name).strip()
    # 공백 정규화
    name = ' '.join(name.split())
    return name.lower()

def normalize_team_name(team_name):
    """팀 이름 정규화 및 매핑"""
    if pd.isna(team_name):
        return ""
    team_name = str(team_name).strip()
    
    # 유니코드 문자를 ASCII로 변환 (é -> e, ü -> u 등)
    # NFD (Normalization Form Decomposed)로 분해한 후, combining characters 제거
    team_name = unicodedata.normalize('NFD', team_name)
    team_name = ''.join(c for c in team_name if unicodedata.category(c) != 'Mn')

    
    # 매핑에서 찾기 (Cell 8에서 정의된 team_name_mapping 사용)
    # 특수 케이스: "Deportivo Alavés" -> "Alaves"로 통일
    # "Deportivo la Coruña" -> "Deportivo La Coruna"로 통일
    team_name_lower = team_name.lower()
    if 'deportivo' in team_name_lower and 'alav' in team_name_lower:
        team_name = 'Alaves'
    elif 'deportivo' in team_name_lower and 'coru' in team_name_lower:
        team_name = 'Deportivo La Coruna'
    
    team_name_mapping = {
        'AFC Bournemouth': 'Bournemouth',
        'Brighton & Hove Albion': 'Brighton',
        'Brighton and Hove Albion': 'Brighton',
        'Tottenham Hotspur': 'Tottenham',
        'West Ham United': 'West Ham',
        'Wolverhampton Wanderers': 'Wolves',
        'Leicester City': 'Leicester',
        'Newcastle United': 'Newcastle',
        'Manchester United': 'Man United',
        'Manchester City': 'Man City',
        'Atlético Madrid': 'Atletico Madrid',
        'Atletico Madrid': 'Atletico Madrid',
        'Celta de Vigo': 'Celta Vigo',
        'Celta Vigo': 'Celta Vigo',
        'Deportivo La Coruña': 'Deportivo La Coruna',
        'Deportivo La Coruna': 'Deportivo La Coruna',
        'Deportivo la Coruña': 'Deportivo La Coruna',
        'Deportivo la Coruna': 'Deportivo La Coruna',
        'Málaga': 'Malaga',
        'Malaga': 'Malaga',
        'Leganés': 'Leganes',
        'Leganes': 'Leganes',
        'Alavés': 'Alaves',
        'Alaves': 'Alaves',
        'Deportivo Alavés': 'Alaves',
        'Deportivo Alaves': 'Alaves',
        'Internazionale': 'Inter',
        'Inter': 'Inter',
        'AC Milan': 'Milan',
        'Hellas Verona': 'Verona',
        'Verona': 'Verona',
        'Bayern München': 'Bayern Munich',
        'Bayern Munich': 'Bayern Munich',
        'Borussia M\'gladbach': 'Borussia Monchengladbach',
        'Borussia Monchengladbach': 'Borussia Monchengladbach',
        'Mönchengladbach': 'Borussia Monchengladbach',
        'Schalke 04': 'Schalke',
        'Schalke': 'Schalke',
        'Hamburger SV': 'Hamburg',
        'Hamburg': 'Hamburg',
        'Mainz 05': 'Mainz',
        'Mainz': 'Mainz',
        'Hannover 96': 'Hannover',
        'Hannover': 'Hannover',
        'Köln': 'Koln',
        'Koln': 'Koln',
        'Paris Saint-Germain': 'Paris Saint-Germain',
        'PSG': 'Paris Saint-Germain',
        'Saint-Étienne': 'Saint-Etienne',
        'Saint-Etienne': 'Saint-Etienne',
        'Amiens SC': 'Amiens',
        'Amiens': 'Amiens',
    }
    
    if team_name in team_name_mapping:
        team_name = team_name_mapping[team_name]
    
    # 공백 정규화 및 소문자 변환
    team_name = ' '.join(team_name.split())
    return team_name.lower()

# Stats 데이터의 팀 이름도 정규화
stats_df_normalized = stats_df.copy()
stats_df_normalized['team_normalized'] = stats_df_normalized['team'].apply(
    lambda x: normalize_team_name(x) if pd.notna(x) else x
)

# Stats 데이터에서 선수별 집계 (이름, 국적, 팀, 경기 수)
# 팀별로 그룹화하기 위해 먼저 선수-팀 조합으로 집계
stats_player_team = stats_df_normalized.groupby(['player', 'team_normalized']).agg({
    'nation': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0],  # 가장 많이 나타난 국적
    'game': 'nunique'  # 경기 수
}).reset_index()
stats_player_team.columns = ['player', 'team', 'stats_nation', 'stats_games']

# 선수별로 모든 팀과 국적 정보 수집 (정규화된 팀 이름 사용)
stats_player_all = stats_df_normalized.groupby('player').agg({
    'team_normalized': lambda x: list(x.unique()),  # 모든 팀 목록 (정규화됨)
    'nation': lambda x: list(x.unique()),  # 모든 국적 목록
    'game': 'nunique'  # 경기 수
}).reset_index()
stats_player_all.columns = ['player', 'stats_teams', 'stats_nations', 'stats_games']

# VAEP 데이터 준비
vaep_comparison = vaep_df[['playerId', 'player_name', 'nation', 'team', 'matches_played']].copy()
vaep_comparison.columns = ['playerId', 'vaep_name', 'vaep_nation', 'vaep_team', 'vaep_games']

# 이름이 같지만 팀이 다른 선수들
name_match_team_diff = []
# 이름이 같지만 국적이 다른 선수들
name_match_nation_diff = []

for _, vaep_row in vaep_comparison.iterrows():
    vaep_name = vaep_row['vaep_name']
    vaep_team = vaep_row['vaep_team']
    vaep_nation = vaep_row['vaep_nation']
    
    if pd.isna(vaep_name):
        continue
    
    vaep_name_norm = normalize_name(vaep_name)
    vaep_team_norm = normalize_team_name(vaep_team) if pd.notna(vaep_team) else ""
    
    # Stats 데이터에서 같은 이름 찾기
    matching_stats = stats_player_all[stats_player_all['player'].apply(lambda x: normalize_name(x) == vaep_name_norm)]
    
    if len(matching_stats) > 0:
        for _, stats_row in matching_stats.iterrows():
            stats_name = stats_row['player']
            stats_teams = stats_row['stats_teams'] if isinstance(stats_row['stats_teams'], list) else []
            stats_nations = stats_row['stats_nations'] if isinstance(stats_row['stats_nations'], list) else []
            
            # 팀이 다른 경우 확인
            if vaep_team_norm:
                # Stats 데이터의 모든 팀과 비교
                stats_teams_norm = [normalize_team_name(t) for t in stats_teams]
                if vaep_team_norm not in stats_teams_norm:
                    # 이름은 같지만 팀이 다른 경우
                    name_match_team_diff.append({
                        'playerId': vaep_row['playerId'],
                        'vaep_name': vaep_name,
                        'stats_name': stats_name,
                        'vaep_team': vaep_team,
                        'stats_teams': ', '.join(stats_teams[:3]) if len(stats_teams) > 0 else 'N/A',
                        'vaep_nation': vaep_nation if pd.notna(vaep_nation) else 'N/A',
                        'stats_nations': ', '.join(stats_nations[:3]) if len(stats_nations) > 0 else 'N/A',
                        'vaep_games': int(vaep_row['vaep_games']) if pd.notna(vaep_row['vaep_games']) else 'N/A',
                        'stats_games': int(stats_row['stats_games']) if pd.notna(stats_row['stats_games']) else 'N/A'
                    })
            
            # 국적이 다른 경우 확인
            if pd.notna(vaep_nation):
                vaep_nation_upper = str(vaep_nation).strip().upper()
                stats_nations_upper = [str(n).strip().upper() for n in stats_nations]
                if vaep_nation_upper not in stats_nations_upper:
                    # 이름은 같지만 국적이 다른 경우
                    name_match_nation_diff.append({
                        'playerId': vaep_row['playerId'],
                        'vaep_name': vaep_name,
                        'stats_name': stats_name,
                        'vaep_team': vaep_team if pd.notna(vaep_team) else 'N/A',
                        'stats_teams': ', '.join(stats_teams[:3]) if len(stats_teams) > 0 else 'N/A',
                        'vaep_nation': vaep_nation,
                        'stats_nations': ', '.join(stats_nations[:3]) if len(stats_nations) > 0 else 'N/A',
                        'vaep_games': int(vaep_row['vaep_games']) if pd.notna(vaep_row['vaep_games']) else 'N/A',
                        'stats_games': int(stats_row['stats_games']) if pd.notna(stats_row['stats_games']) else 'N/A'
                    })

team_diff_df = pd.DataFrame(name_match_team_diff)
nation_diff_df = pd.DataFrame(name_match_nation_diff)

# 중복 제거 (같은 선수가 여러 번 나타날 수 있음)
if len(team_diff_df) > 0:
    team_diff_df = team_diff_df.drop_duplicates(subset=['playerId', 'stats_name'])
if len(nation_diff_df) > 0:
    nation_diff_df = nation_diff_df.drop_duplicates(subset=['playerId', 'stats_name'])

# 결과 출력
print("이름은 같지만 팀이 다른 선수들")
if len(team_diff_df) > 0:
    team_diff_df = team_diff_df.sort_values('vaep_name')
    team_display_df = team_diff_df[['playerId', 'vaep_name', 'stats_name', 'vaep_team', 'stats_teams', 
                                     'vaep_nation', 'stats_nations', 'vaep_games', 'stats_games']].copy()
    team_display_df.columns = ['Player ID', 'VAEP 이름', 'Stats 이름', 'VAEP 팀', 'Stats 팀 목록', 
                               'VAEP 국적', 'Stats 국적 목록', 'VAEP 경기', 'Stats 경기']
    print(f"총 {len(team_display_df)}명")
    display(team_display_df)
else:
    print("해당하는 선수가 없습니다.")

print("\n이름은 같지만 국적이 다른 선수들")
if len(nation_diff_df) > 0:
    nation_diff_df = nation_diff_df.sort_values('vaep_name')
    nation_display_df = nation_diff_df[['playerId', 'vaep_name', 'stats_name', 'vaep_team', 'stats_teams', 
                                         'vaep_nation', 'stats_nations', 'vaep_games', 'stats_games']].copy()
    nation_display_df.columns = ['Player ID', 'VAEP 이름', 'Stats 이름', 'VAEP 팀', 'Stats 팀 목록', 
                                  'VAEP 국적', 'Stats 국적 목록', 'VAEP 경기', 'Stats 경기']
    print(f"총 {len(nation_display_df)}명")
    display(nation_display_df)
else:
    print("해당하는 선수가 없습니다.")


이름은 같지만 팀이 다른 선수들
총 46명


,Player ID,VAEP 이름,Stats 이름,VAEP 팀,Stats 팀 목록,VAEP 국적,Stats 국적 목록,VAEP 경기,Stats 경기
37,214220,Alisson,Alisson,liverpool,roma,GER,BRA,42,37
7,3541,Bruno,Bruno,villarreal,brighton,ESP,ESP,2,25
17,5677,Bruno,Bruno,getafe,brighton,ESP,ESP,18,25
9,3645,Chory Castro,Chory Castro,m\u00e1laga,malaga,ESP,URU,26,26
3,3322,Cristiano Ronaldo,Cristiano Ronaldo,juventus,real madrid,POR,POR,38,27
1,340,Cuco Martina,Cuco Martina,stoke,everton,Cura\u00e7ao,CUW,21,21
26,20583,Danilo,Danilo,udinese,man city,ITA,BRA,32,23
14,4338,Diego Costa,Diego Costa,atl\u00e9tico madrid,atletico madrid,ESP,ESP,19,15
36,169002,Diego Rico,Diego Rico,legan\u00e9s,leganes,ESP,ESP,26,26
27,20928,Felipe,Felipe,spal,hannover,ITA,BRA,30,6



이름은 같지만 국적이 다른 선수들
총 40명


,Player ID,VAEP 이름,Stats 이름,VAEP 팀,Stats 팀 목록,VAEP 국적,Stats 국적 목록,VAEP 경기,Stats 경기
31,214220,Alisson,Alisson,liverpool,roma,GER,BRA,42,37
23,40726,Allan,Allan,napoli,napoli,POR,BRA,38,38
5,3692,Bernardo Espinosa,Bernardo Espinosa,girona,girona,ESP,COL,34,34
35,265673,Bernardo Silva,Bernardo Silva,man city,man city,FRA,POR,39,35
33,222766,Bruno Gaspar,Bruno Gaspar,fiorentina,fiorentina,POR,ANG,15,15
38,385273,Charles,Charles,eibar,eibar,ESP,BRA,30,30
4,3645,Chory Castro,Chory Castro,m\u00e1laga,malaga,ESP,URU,26,26
0,340,Cuco Martina,Cuco Martina,stoke,everton,Cura\u00e7ao,CUW,21,21
2,3334,Dani Alves,Dani Alves,paris saint-germain,paris saint-germain,ESP,BRA,25,25
15,20583,Danilo,Danilo,udinese,man city,ITA,BRA,32,23
